In [1]:
import sys
test_input = "../test_data/diann_test_input.tsv"




In [2]:
import refquant.utils.table_import as table_import
import pandas as pd
df_diann = pd.read_csv(test_input, sep="\t")
reformatter = table_import.TableReformatterDIANN(test_input, quantitative_extraction_types = [ "diann_precursors_mDIA"])

input file
/Users/constantin/workspace/refquant/test_data
../test_data/diann_test_input.tsv
using input type diann_precursors_mDIA


In [3]:
import numpy as np
def get_precursor_df_from_diann(df_diann):
    df_precursor = df_diann[["Run", "Precursor.Translated", 'Stripped.Sequence', 'Precursor.Charge', "Precursor.Id"]].drop_duplicates()
    df_precursor = df_precursor[[x>0 for x in df_precursor["Precursor.Translated"]]]

    df_precursor["channel"] = df_precursor["Precursor.Id"].apply(get_channel_name)
    df_precursor['sequence_charge'] = df_precursor["Stripped.Sequence"] + "_" + df_precursor["Precursor.Charge"].astype(str)
    df_precursor = filter_for_channel_consistency(df_precursor)
    df_precursor = df_precursor[["sequence_charge", "Run", "Precursor.Translated", "channel"]].drop_duplicates()
    
    return df_precursor

def get_channel_name(precursor_id):
    return precursor_id.split(")")[0]

def filter_for_channel_consistency(df_precursor):
    selected_precurso_run_combos = []
    df_precursor = df_precursor.set_index(["Run", "sequence_charge"])
    run_precursors = set(df_precursor.index)
    for run_precursor in run_precursors:
        precursor_df = df_precursor.loc[run_precursor]
        if "(Dimethyl-n-4" in precursor_df["channel"].values or "(Dimethyl-n-8" in precursor_df["channel"].values:
            selected_precurso_run_combos.append(run_precursor)

    return df_precursor.loc[selected_precurso_run_combos].reset_index()




def get_annotated_reformated_df(df_reformatted):
    df_precursor = df_reformatted
    df_precursor["stripped_sequence"] = df_precursor["precursor"].apply(lambda x: x.split("_")[1])
    df_precursor["charge"] = df_precursor["precursor"].apply(lambda x: x.split("_")[-2])
    df_precursor['sequence_charge'] = df_precursor["stripped_sequence"] + "_" + df_precursor["charge"].astype(str)
    return df_precursor

diann_precursor_df = get_precursor_df_from_diann(df_diann)
refomatted_annoated_df = get_annotated_reformated_df(reformatter.diann_table_reformatted)

display(diann_precursor_df)
display(refomatted_annoated_df)


def check_that_sequence_charge_are_the_same(diann_precursor_df, refomatted_annoated_df):
    sequence_charge_diann = set(diann_precursor_df["sequence_charge"])
    sequence_charge_reformatted = set(refomatted_annoated_df["sequence_charge"])
    assert sequence_charge_diann == sequence_charge_reformatted
    print("sequence_charge are the same")

def check_that_sequence_charge_run_are_the_same(diann_precursor_df, refomatted_annoated_df):
    sequence_charge_run_diann = set(zip(diann_precursor_df["sequence_charge"], diann_precursor_df["Run"]))
    sequence_charge_run_reformatted = set(zip(refomatted_annoated_df["sequence_charge"], refomatted_annoated_df["run"]))
    assert sequence_charge_run_diann == sequence_charge_run_reformatted
    print("sequence_charge_run are the same")



def check_that_sequence_charge_run_channel_intensities_are_the_same(diann_precursor_df, refomatted_annoated_df):
    #filter to channel 4
    diann_precursor_df = diann_precursor_df[["-n-4" in x for x in diann_precursor_df["channel"]]]
    refomatted_annoated_df = refomatted_annoated_df[[x> 0 for x in refomatted_annoated_df["target4"]]]

    #create dicts sequence_charge_run2intensity
    sequence_charge_run_diann = dict(zip(zip(diann_precursor_df["sequence_charge"], diann_precursor_df["Run"]), diann_precursor_df["Precursor.Translated"]))
    sequence_charge_run_channel = dict(zip(zip(refomatted_annoated_df["sequence_charge"], refomatted_annoated_df["run"]), refomatted_annoated_df["target4"]))
    
    #assert that keys are the same
    assert set(sequence_charge_run_diann.keys()) == set(sequence_charge_run_channel.keys())
    #assert that values are the same
    for key in sequence_charge_run_diann.keys():
        assert np.isclose(sequence_charge_run_diann[key], sequence_charge_run_channel[key])

    print("values in channel 4 are the same")

check_that_sequence_charge_are_the_same(diann_precursor_df, refomatted_annoated_df)
check_that_sequence_charge_run_are_the_same(diann_precursor_df, refomatted_annoated_df)
check_that_sequence_charge_run_channel_intensities_are_the_same(diann_precursor_df, refomatted_annoated_df)

/var/folders/wb/j08kzzbd2hd8838tn3ll0jfm0000gn/T/ipykernel_74810/4231689626.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
  precursor_df = df_precursor.loc[run_precursor]


,sequence_charge,Run,Precursor.Translated,channel
0,IATTTASAATAAAIGATPR_3,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,4497.572754,(Dimethyl-n-0
1,IATTTASAATAAAIGATPR_3,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,663.588501,(Dimethyl-n-4
2,SDPIMLLK_2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,20311.923830,(Dimethyl-n-0
3,SDPIMLLK_2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,495.055847,(Dimethyl-n-8
4,DPQALSEHLK_3,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,10005.327150,(Dimethyl-n-0
...,...,...,...,...
20774,TLQEVTEMDSVK_3,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,2444.139404,(Dimethyl-n-8
20775,AVAALLTIPEAEK_2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,18766.039060,(Dimethyl-n-0
20776,AVAALLTIPEAEK_2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,72.392883,(Dimethyl-n-8
20777,FQELLQDLEK_2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,17013.021480,(Dimethyl-n-0


channel,run,protein,precursor,ion,reference,target4,target8,stripped_sequence,charge,sequence_charge
1,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,A1X283,SEQ_AAAASVPNADGLK_MOD_AAAASVPNADGLK_CHARGE_3_,SEQ_AAAASVPNADGLK_MOD_AAAASVPNADGLK_CHARGE_3_,104656.578100,16279.777340,10646.807620,AAAASVPNADGLK,3,AAAASVPNADGLK_3
2,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,A1X283,SEQ_AAAASVPNADGLK_MOD_AAAASVPNADGLK_CHARGE_4_,SEQ_AAAASVPNADGLK_MOD_AAAASVPNADGLK_CHARGE_4_,6812.289063,0.000000,2191.179199,AAAASVPNADGLK,4,AAAASVPNADGLK_4
3,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,A1X283,SEQ_EGWAPASYLK_MOD_EGWAPASYLK_CHARGE_2_,SEQ_EGWAPASYLK_MOD_EGWAPASYLK_CHARGE_2_,7365.604004,1033.218262,807.697388,EGWAPASYLK,2,EGWAPASYLK_2
4,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,A1X283,SEQ_EGWAPASYLK_MOD_EGWAPASYLK_CHARGE_3_,SEQ_EGWAPASYLK_MOD_EGWAPASYLK_CHARGE_3_,5469.298340,5300.778809,7401.166504,EGWAPASYLK,3,EGWAPASYLK_3
10,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,A1X283,SEQ_SFLPGEGPGR_MOD_SFLPGEGPGR_CHARGE_2_,SEQ_SFLPGEGPGR_MOD_SFLPGEGPGR_CHARGE_2_,6025.197266,76.155418,898.278259,SFLPGEGPGR,2,SFLPGEGPGR_2
...,...,...,...,...,...,...,...,...,...,...
9857,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,Q9Y4H2,SEQ_TDSLAATPPAAK_MOD_TDSLAATPPAAK_CHARGE_3_,SEQ_TDSLAATPPAAK_MOD_TDSLAATPPAAK_CHARGE_3_,21801.226560,0.000000,29967.902340,TDSLAATPPAAK,3,TDSLAATPPAAK_3
9858,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,Q9Y4H2,SEQ_TDSLAATPPAAK_MOD_TDSLAATPPAAK_CHARGE_4_,SEQ_TDSLAATPPAAK_MOD_TDSLAATPPAAK_CHARGE_4_,0.000000,23015.373050,35270.035160,TDSLAATPPAAK,4,TDSLAATPPAAK_4
9860,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,Q9Y4H2,SEQ_VALLPAGGALQHSR_MOD_VALLPAGGALQHSR_CHARGE_4_,SEQ_VALLPAGGALQHSR_MOD_VALLPAGGALQHSR_CHARGE_4_,661.620666,0.000000,8.552920,VALLPAGGALQHSR,4,VALLPAGGALQHSR_4
9861,20220730_TIMS06_MCT_SA_HeLa_whi40_M07_Ref0s4s8...,Q9Y4H2,SEQ_VSGDAAQDLDR_MOD_VSGDAAQDLDR_CHARGE_2_,SEQ_VSGDAAQDLDR_MOD_VSGDAAQDLDR_CHARGE_2_,1075.352295,555.666870,1463.276367,VSGDAAQDLDR,2,VSGDAAQDLDR_2


sequence_charge are the same
sequence_charge_run are the same
values in channel 4 are the same
